In [26]:
import sys, os
import numpy as np
import math

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch

import math
import matplotlib.pyplot as plt

from argparse import Namespace

from FRTrain_arch import Generator, DiscriminatorF, DiscriminatorR, weights_init_normal, test_model

import warnings
warnings.filterwarnings("ignore")


# Autoreload in case that the custom modules are changed
%reload_ext autoreload
%autoreload 2

In [27]:
print(Generator())
print(DiscriminatorR())
print(DiscriminatorF())

Generator(
  (model): Sequential(
    (0): Linear(in_features=3, out_features=1, bias=True)
  )
)
DiscriminatorR(
  (model): Sequential(
    (0): Linear(in_features=4, out_features=16, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=16, out_features=1, bias=True)
    (3): Sigmoid()
  )
)
DiscriminatorF(
  (model): Sequential(
    (0): Linear(in_features=1, out_features=1, bias=True)
    (1): Sigmoid()
  )
)


### Load and process real data from Adult Cencus dataset ( without crowdsourcing validation set)

In [42]:
# a namespace object which contains some of the hyperparameters
opt = Namespace(num_train=5000, num_val1=500, num_val2=0, num_test=1000, in_feature = 6)

In [29]:
# num_train = opt.num_train
# num_val1 = opt.num_val1
# num_val2 = opt.num_val2
# num_test = opt.num_test

# XS = np.load('adultCencus_data/XS_adult.npy', allow_pickle=True) # Input features
# y = np.load('adultCencus_data/y_adult.npy', allow_pickle=True) # Labels
# s1 = np.load('adultCencus_data/s_adult.npy', allow_pickle=True) # Sensitive features
# s1 = s1.astype(float)

# XS = torch.FloatTensor(XS)
# y = torch.FloatTensor(y)
# s1 = torch.FloatTensor(s1)

# XS_train = XS[:num_train - num_val1]
# y_train = y[:num_train - num_val1]
# s1_train = s1[:num_train - num_val1]

# # 10% val set size
# XS_val = XS[num_train: num_train + num_val1]
# y_val = y[num_train: num_train + num_val1]
# s1_val = s1[num_train: num_train + num_val1]

# XS_test = XS[num_train + num_val1 + num_val2 : num_train + num_val1 + num_val2 + num_test]
# y_test = y[num_train + num_val1 + num_val2 : num_train + num_val1 + num_val2 + num_test]
# s1_test = s1[num_train + num_val1 + num_val2 : num_train + num_val1 + num_val2 + num_test]

### Load and process real data from Adult Cencus dataset ( with crowdsourcing validation set)

In [30]:

# opt = Namespace(num_train=1800, num_val=200, num_test=1000)

# num_train = opt.num_train
# num_val = opt.num_val
# num_test = opt.num_test

# XS = np.load('adultCencus_data/train/XS_train_adult.npy', allow_pickle=True) # Input features
# y = np.load('adultCencus_data/train/y_train_adult.npy', allow_pickle=True) # Labels
# s1 = np.load('adultCencus_data/train/s_train_adult.npy', allow_pickle=True) # Sensitive features
# s1 = s1.astype(float)

# XS = torch.FloatTensor(XS)
# y = torch.FloatTensor(y)
# s1 = torch.FloatTensor(s1)

# XS_train = XS[:num_train]
# y_train = y[:num_train] 
# s1_train = s1[:num_train]


# XS_test = XS[num_train: num_train + num_test]
# y_test = y[num_train: num_train + num_test]
# s1_test = s1[num_train: num_train + num_test]

# # 10% validation set constructing from crowdsourcing data
# XS_val = np.load('adultCencus_data/val/XS_val_adult.npy', allow_pickle=True)
# y_val = np.load('adultCencus_data/val/y_val_adult.npy', allow_pickle=True)
# s1_val = np.load('adultCencus_data/val/s_val_adult.npy', allow_pickle=True)
# s1_val = s1_val.astype(float)


# XS_val = torch.FloatTensor(XS_val)
# y_val = torch.FloatTensor(y_val)
# s1_val = torch.FloatTensor(s1_val)

# XS_val = XS_val[:num_val]
# y_val = y_val[:num_val]
# s1 = s1[:num_val]




### Load and process real data from COMPAS dataset

In [50]:
num_train = opt.num_train
num_val1 = opt.num_val1
num_val2 = opt.num_val2
num_test = opt.num_test

XS = np.load('compas_data/XS_compas.npy', allow_pickle=True) # Input features
y = np.load('compas_data/y_compas.npy', allow_pickle=True) # Labels
s1 = np.load('compas_data/s_compas.npy', allow_pickle=True) # Sensitive features
s1 = s1.astype(float)

XS = torch.FloatTensor(XS)
y = torch.FloatTensor(y)
s1 = torch.FloatTensor(s1)

XS_train = XS[:num_train - num_val1]
y_train = y[:num_train - num_val1]
s1_train = s1[:num_train - num_val1]

# 10% val set size
XS_val = XS[num_train - num_val1: num_train]
y_val = y[num_train - num_val1: num_train]
s1_val = s1[num_train - num_val1: num_train]

XS_test = XS[num_train : num_train + num_test]
y_test = y[num_train : num_train + num_test]
s1_test = s1[num_train : num_train + num_test]

In [51]:
print("XS train", XS_train.shape)
print("y train", y_train.shape)
print("s1 train", s1_train.shape)

print("------------------------------")

print("XS val", XS_val.shape)
print("y val", y_val.shape)
print("s1 val", s1_val.shape)

print("------------------------------")


print("XS test", XS_test.shape)
print("y test", y_test.shape)
print("s1 test", s1_test.shape)

# print(XS_train)
# print(y_train)
# print(s1_train)

XS train torch.Size([4500, 6])
y train torch.Size([4500])
s1 train torch.Size([4500])
------------------------------
XS val torch.Size([500, 6])
y val torch.Size([500])
s1 val torch.Size([500])
------------------------------
XS test torch.Size([1000, 6])
y test torch.Size([1000])
s1 test torch.Size([1000])


In [52]:
print("--------------------- Number of Data -------------------------" )
print(
    "Train data : %d, Validation data : %d, Test data : %d "
    % (len(y_train), len(y_val), len(y_test))
)       
print("--------------------------------------------------------------")

--------------------- Number of Data -------------------------
Train data : 4500, Validation data : 500, Test data : 1000 
--------------------------------------------------------------


### Model Training

In [34]:
def train_model(train_tensors, val_tensors, test_tensors, train_opt, lambda_f, lambda_r, seed):
    """
      Trains FR-Train by using the classes in FRTrain_arch.py.
      
      Args:
        train_tensors: Training data.
        val_tensors: Clean validation data.
        test_tensors: Test data.
        train_opt: Options for the training. It currently contains size of validation set, 
                number of epochs, generator/discriminator update ratio, and learning rates.
        lambda_f: The tuning knob for L_2 (ref: FR-Train paper, Section 3.3).
        lambda_r: The tuning knob for L_3 (ref: FR-Train paper, Section 3.3).
        seed: An integer value for specifying torch random seed.
        
      Returns:
        Information about the tuning knobs (lambda_f, lambda_r),
        the test accuracy of the trained model, and disparate impact of the trained model.
    """
    
    XS_train = train_tensors.XS_train
    y_train = train_tensors.y_train
    s1_train = train_tensors.s1_train
    
    XS_val = val_tensors.XS_val
    y_val = val_tensors.y_val
    s1_val = val_tensors.s1_val
    
    XS_test = test_tensors.XS_test
    y_test = test_tensors.y_test
    s1_test = test_tensors.s1_test
    
    # Saves return values here
    test_result = []
    
    val = train_opt.val # Number of data points in validation set
    k = train_opt.k     # Updates ratio of generator and discriminator (1:k training).
    n_epochs = train_opt.n_epochs  # Number of training epoch
    
    # Changes the input validation data to an appropriate shape for the training
    XSY_val = torch.cat([XS_val, y_val.reshape((y_val.shape[0], 1))], dim=1)  

    # The loss values of each component will be saved in the following lists. 
    # We can draw epoch-loss graph by the following lists, if necessary.
    g_losses =[]
    d_f_losses = []
    d_r_losses = []
    clean_test_result = []

    bce_loss = torch.nn.BCELoss()

    # Initializes generator and discriminator
    generator = Generator(in_feat=opt.in_feature)
    discriminator_F = DiscriminatorF()

    discriminator_R = DiscriminatorR(in_feat = opt.in_feature + 1)

    # Initializes weights
    torch.manual_seed(seed)
    generator.apply(weights_init_normal)
    discriminator_F.apply(weights_init_normal)
    discriminator_R.apply(weights_init_normal)

    optimizer_G = torch.optim.Adam(generator.parameters(), lr=train_opt.lr_g)
    optimizer_D_F = torch.optim.SGD(discriminator_F.parameters(), lr=train_opt.lr_f)
    optimizer_D_R = torch.optim.SGD(discriminator_R.parameters(), lr=train_opt.lr_r)

    XSY_val_data = XSY_val[:val]

    train_len = XS_train.shape[0]
    val_len = XSY_val.shape[0]

    # Ground truths using in Disriminator_R
    Tensor = torch.FloatTensor
    valid = Variable(Tensor(train_len, 1).fill_(1.0), requires_grad=False)
    generated = Variable(Tensor(train_len, 1).fill_(0.0), requires_grad=False)
    fake = Variable(Tensor(train_len, 1).fill_(0.0), requires_grad=False)
    clean = Variable(Tensor(val_len, 1).fill_(1.0), requires_grad=False)
    
    r_weight = torch.ones_like(y_train, requires_grad=False).float()
    r_ones = torch.ones_like(y_train, requires_grad=False).float()

    for epoch in range(n_epochs):

        # -------------------
        #  Forwards Generator
        # -------------------
        if epoch % k == 0 or epoch < 500:
            optimizer_G.zero_grad()

        gen_y = generator(XS_train)
        gen_data = torch.cat([XS_train, gen_y.detach().reshape((gen_y.shape[0], 1))], dim=1)
        
        # -----------------------------
        #  Trains Fairness Discriminator
        # -----------------------------

        optimizer_D_F.zero_grad()

        # Discriminator_F tries to distinguish the sensitive groups by using the output of the generator.
        d_f_loss= bce_loss(discriminator_F(gen_y.detach()), s1_train.unsqueeze(1))
        d_f_loss.backward()
        d_f_losses.append(d_f_loss)
        optimizer_D_F.step()
            

        # ---------------------------------
        #  Trains Robustness Discriminator
        # ---------------------------------
        optimizer_D_R.zero_grad()

        # Discriminator_R tries to distinguish whether the input is from the validation data or the generated data from generator.
        clean_loss = bce_loss(discriminator_R(XSY_val_data), clean)
        poison_loss = bce_loss(discriminator_R(gen_data.detach()), fake)
        d_r_loss = 0.5 * (clean_loss + poison_loss)

        d_r_loss.backward()
        d_r_losses.append(d_r_loss)
        optimizer_D_R.step()


        # ---------------------
        #  Updates Generator
        # ---------------------

        # Loss measures generator's ability to fool the discriminators
        if epoch < 500 :
            g_loss = bce_loss((F.tanh(gen_y)+1)/2, (y_train.unsqueeze(1)+1)/2)
            g_loss.backward()
            g_losses.append(g_loss)
            optimizer_G.step()
    
        elif epoch % k == 0:
            r_decision = discriminator_R(gen_data)
            r_gen = bce_loss(r_decision, generated)
            
            # ------------------------------
            #  Re-weights using output of D_R
            # ------------------------------
            
            if epoch % 100 == 0:
                loss_ratio = (g_losses[-1]/d_r_losses[-1]).detach()
                a = 1/(1+torch.exp(-(loss_ratio-3)))
                b = 1-a
                r_weight_tmp = r_decision.detach().squeeze()
                r_weight = a * r_weight_tmp + b * r_ones

            f_cost = F.binary_cross_entropy(discriminator_F(gen_y), s1_train.unsqueeze(1), reduction="none").squeeze()
            g_cost = F.binary_cross_entropy_with_logits(gen_y.squeeze(), (y_train.squeeze()+1)/2, reduction="none").squeeze()

            f_gen = torch.mean(f_cost*r_weight)
            g_loss = (1-lambda_f-lambda_r) * torch.mean(g_cost*r_weight) -  lambda_r * r_gen - lambda_f * f_gen

            g_loss.backward()
            optimizer_G.step()

        g_losses.append(g_loss)

        if epoch % 200 == 0:
            print(
                "[Lambda_f: %1f] [Epoch %d/%d] [D_F loss: %f] [D_R loss: %f] [G loss: %f]"
                % (lambda_f, epoch, n_epochs, d_f_losses[-1], d_r_losses[-1], g_losses[-1])
            )

#     torch.save(generator.state_dict(), './FR-Train_on_clean_synthetic.pth')
    tmp = test_model(generator, XS_test, y_test, s1_test)
    test_result.append([lambda_f, lambda_r, tmp[0].item(), tmp[1]])

    return test_result

In [53]:
train_result = []
train_tensors = Namespace(XS_train = XS_train, y_train = y_train, s1_train = s1_train)
val_tensors = Namespace(XS_val = XS_val, y_val = y_val, s1_val = s1_val) 
test_tensors = Namespace(XS_test = XS_test, y_test = y_test, s1_test = s1_test)

train_opt = Namespace(val=len(y_val), n_epochs=4000, k=3, lr_g=0.005, lr_f=0.01, lr_r=0.001)      
seed = 1 

lambda_f_set = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.85] # Lambda value for the fairness discriminator of FR-Train.
lambda_r = 0.1 # Lambda value for the robustness discriminator of FR-Train.

for lambda_f in lambda_f_set:
    train_result.append(train_model(train_tensors, val_tensors, test_tensors, train_opt, lambda_f = lambda_f, lambda_r = lambda_r, seed=seed))

[Lambda_f: 0.200000] [Epoch 0/4000] [D_F loss: 0.690735] [D_R loss: 0.716739] [G loss: 0.758188]
[Lambda_f: 0.200000] [Epoch 200/4000] [D_F loss: 0.672830] [D_R loss: 0.708732] [G loss: 0.607296]
[Lambda_f: 0.200000] [Epoch 400/4000] [D_F loss: 0.656372] [D_R loss: 0.706759] [G loss: 0.607182]
[Lambda_f: 0.200000] [Epoch 600/4000] [D_F loss: 0.644991] [D_R loss: 0.704101] [G loss: 0.224391]
[Lambda_f: 0.200000] [Epoch 800/4000] [D_F loss: 0.638044] [D_R loss: 0.703087] [G loss: 0.221707]
[Lambda_f: 0.200000] [Epoch 1000/4000] [D_F loss: 0.638369] [D_R loss: 0.702254] [G loss: 0.220420]
[Lambda_f: 0.200000] [Epoch 1200/4000] [D_F loss: 0.638912] [D_R loss: 0.701391] [G loss: 0.219682]
[Lambda_f: 0.200000] [Epoch 1400/4000] [D_F loss: 0.639116] [D_R loss: 0.700513] [G loss: 0.219268]
[Lambda_f: 0.200000] [Epoch 1600/4000] [D_F loss: 0.639156] [D_R loss: 0.699627] [G loss: 0.219063]
[Lambda_f: 0.200000] [Epoch 1800/4000] [D_F loss: 0.639149] [D_R loss: 0.698749] [G loss: 0.218939]
[Lambda

In [54]:
print("--------------------------------------------------------------------------------")
print("------------------ Training Results of FR-Train on clean data ------------------" )
for i in range(len(train_result)):
    print(
        "[Lambda_f: %.2f] [Lambda_r: %.2f] Accuracy : %.3f, Disparate Impact : %.3f "
        % (train_result[i][0][0], train_result[i][0][1], train_result[i][0][2], train_result[i][0][3])
    )       
print("--------------------------------------------------------------------------------")

--------------------------------------------------------------------------------
------------------ Training Results of FR-Train on clean data ------------------
[Lambda_f: 0.20] [Lambda_r: 0.10] Accuracy : 0.675, Disparate Impact : 0.926 
[Lambda_f: 0.30] [Lambda_r: 0.10] Accuracy : 0.672, Disparate Impact : 0.967 
[Lambda_f: 0.40] [Lambda_r: 0.10] Accuracy : 0.668, Disparate Impact : 0.929 
[Lambda_f: 0.50] [Lambda_r: 0.10] Accuracy : 0.668, Disparate Impact : 0.872 
[Lambda_f: 0.60] [Lambda_r: 0.10] Accuracy : 0.666, Disparate Impact : 0.839 
[Lambda_f: 0.70] [Lambda_r: 0.10] Accuracy : 0.664, Disparate Impact : 0.820 
[Lambda_f: 0.80] [Lambda_r: 0.10] Accuracy : 0.646, Disparate Impact : 0.578 
[Lambda_f: 0.85] [Lambda_r: 0.10] Accuracy : 0.622, Disparate Impact : 0.446 
--------------------------------------------------------------------------------


In [37]:
XS_train.shape

torch.Size([1800, 6])

In [38]:
y_train.shape

torch.Size([1800])

In [39]:
s1_train.shape

torch.Size([1800])

In [40]:
z_0_mask = (s1 == 0.0)
z_0 = int(torch.sum(z_0_mask))
z_0

2103

In [41]:
s1_train

tensor([1., 1., 1.,  ..., 1., 1., 0.])